In [1]:
!pip install transformers datasets peft
!pip install -q -U bitsandbytes
!pip install transformers
# !pip install -q -U git+https://github.com/huggingface/peft.git
!pip install peft
!pip install -q datasets
!pip install -qqq trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 8.4 MB/s eta 0:00:00:00:01


In [17]:
from huggingface_hub import login
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch
from peft import LoraConfig, get_peft_model, PromptTuningConfig
import torch
import transformers


In [3]:
from huggingface_hub import login

huggingface_token = "API Key"
login(token=huggingface_token)

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    device_map="auto", 
    use_auth_token=True
)


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [5]:
import requests

url = "https://huggingface.co/datasets/miladmim/slim-orca-dedup-chat-50k-persian/resolve/main/data.jsonl"
response = requests.get(url)
with open("data.jsonl", "wb") as f:
    f.write(response.content)
    

In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import requests

# Download the dataset file
url = "https://huggingface.co/datasets/miladmim/slim-orca-dedup-chat-50k-persian/resolve/main/data.jsonl"
response = requests.get(url)
with open("data.jsonl", "wb") as f:
    f.write(response.content)

# Fix the JSON Lines file
fixed_lines = []
with open("data.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        try:
            # Parse the JSON object
            json_obj = json.loads(line)

            # Ensure consistent structure
            if not isinstance(json_obj, list) or len(json_obj) < 2:
                continue

            # Extract user and assistant messages
            user_message = None
            assistant_message = None
            for msg in json_obj:
                if msg["role"] == "user":
                    user_message = msg["content"]
                elif msg["role"] == "assistant":
                    assistant_message = msg["content"]

            if user_message is None or assistant_message is None:
                continue

            fixed_obj = {"instruction": user_message, "response": assistant_message}
            fixed_lines.append(fixed_obj)

        except json.JSONDecodeError as e:
            print(f"Error parsing line: {line}")
            print(f"Error: {e}")

# Save the fixed file
with open("fixed_data_consistent.jsonl", "w", encoding="utf-8") as f:
    for obj in fixed_lines:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

# Load the fixed dataset
dataset = load_dataset("json", data_files="fixed_data_consistent.jsonl")

print("Dataset structure:")
print(dataset)

print("\nSample data:")
for example in dataset["train"].select(range(2)):  # Display 2 examples
    print("Instruction:", example["instruction"])
    print("Response:", example["response"])
    print("-" * 50)

train_val_split = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]

# print("Training dataset size:", len(train_dataset))
# print("Validation dataset size:", len(val_dataset) if val_dataset else "No validation set")


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

data.jsonl:   0%|          | 0.00/97.8M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Failed to load JSON from file '/root/.cache/huggingface/hub/datasets--miladmim--slim-orca-dedup-chat-50k-persian/snapshots/cbd7045a43dd3324fb98cf9dc37b9c0d5fcfb0d8/data.jsonl' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


Error loading dataset: An error occurred while generating the dataset
Attempting to manually fix and load the dataset...


Generating train split: 0 examples [00:00, ? examples/s]

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 10000
    })
})

Sample data:
Instruction: فاصله‌ها را اضافه کنید: بازی‌های تراکتور، تراکتور از مزرعه، تراکتور با تریلر
Response: در اینجا عبارات جدا شده با فاصله‌های اضافه شده آمده است:

1. بازی‌های تراکتور
2. تراکتور از مزرعه
3. تراکتور با تریلر

حال اجازه دهید توضیح مفصلی درباره هر عبارت جدا شده ارائه دهم.

1. بازی‌های تراکتور: بازی‌های تراکتور به دسته‌ای از بازی‌های ویدئویی یا آنلاین اشاره دارد که در آن‌ها تراکتورها به عنوان بخش اصلی یا جدایی‌ناپذیر بازی وجود دارند. این بازی‌ها می‌توانند اهداف و مقاصد مختلفی داشته باشند، از جمله مسابقه، شبیه‌سازی کشاورزی یا هدایت تراکتور. بازی‌های تراکتور به طیف وسیعی از افراد، از کودکان که تراکتورها را جذاب می‌دانند، تا بزرگسالانی که از شبیه‌سازی و چالش‌های کار با ماشین‌آلات سنگین لذت می‌برند، جذاب است. برخی از بازی‌های محبوب تراکتور شامل شبیه‌ساز کشاورزی، جنون تراکتور، آزمایش تراکتور و بسیاری دیگر است. این بازی‌ها در پلتفرم‌های م

In [7]:
CUTOFF_LEN = 128 

def tokenize(prompt, tokenizer,add_eos_token=True):
    result = tokenizer(
        prompt+"[PAD]",
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"],
        "attention_mask": result["attention_mask"],
        "labels": result["input_ids"].copy()  # Ensure matching lengths
    }

def generate_prompt(examples):
    return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
###Instruction:
{examples["instruction"]}

###Output:
{examples["response"]}
"""

train_dataset = train_dataset.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))
val_dataset = val_dataset.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
def free_gpu_memory():
    import gc
    gc.collect()
    torch.cuda.empty_cache()

free_gpu_memory()

In [10]:
!pip install evaluate bert-score rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=d6bdf1ddcba1a32f50bed64319decaca51e558d7003e99940a3a78c28446ba14
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate
from bert_score import score

rouge = evaluate.load("rouge")

def generate_predictions(model, tokenizer, input_texts, max_length=100):
    predictions = []
    for text in input_texts:
        inputs = tokenizer(text, return_tensors="pt").to(model.device)
        output = model.generate(**inputs, max_new_tokens=max_length)
        decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(decoded_output)
    return predictions

def evaluate_rouge(predictions, references):
    results = rouge.compute(predictions=predictions, references=references)
    return results

def evaluate_bert_score(predictions, references):
    P, R, F1 = score(predictions, references, lang="en", model_type="bert-base-uncased")
    return {
        "precision": P.mean().item(),
        "recall": R.mean().item(),
        "f1": F1.mean().item()
    }

val_inputs = [generate_prompt(example) for i, example in enumerate(val_dataset) if i < 100]
val_references = [example["response"] for i, example in enumerate(val_dataset) if i < 100]

# Generate predictions
val_predictions = generate_predictions(model, tokenizer, val_inputs, max_length=100)

# Compute ROUGE scores
rouge_scores = evaluate_rouge(val_predictions, val_references)
print("ROUGE Scores:", rouge_scores)

# Compute BERTScore
bert_scores = evaluate_bert_score(val_predictions, val_references)
print("BERTScore:", bert_scores)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:1889: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


ROUGE Results: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTScore Results: {'precision': 0.6466328501701355, 'recall': 0.8137058615684509, 'f1': 0.7206121683120728}


In [13]:
!pip install wandb -qU
!wandb login 
!wandb login --relogin


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 89.8 MB/s eta 0:00:00:00:0100:01
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [14]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [ ]:
free_gpu_memory()

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    device_map="auto", 
    use_auth_token=True
)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

# Define Prompt Tuning configuration
prompt_tuning_config = PromptTuningConfig(
    task_type="CAUSAL_LM",
    num_virtual_tokens=5,
    token_dim=3072,
    num_attention_heads=24,
    num_layers=28,
)

# Apply Prompt Tuning to the model
model = get_peft_model(model, prompt_tuning_config)

# Count trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"Trainable parameters: {trainable_params}")
print(f"Total parameters: {total_params}")
print(f"Percentage of trainable parameters: {100 * trainable_params / total_params:.2f}%")

free_gpu_memory()

training_args = TrainingArguments(
    output_dir="./prompt-tuning-results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=10,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./prompt-tuning-logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    optim="adamw_hf"
)

free_gpu_memory()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False

# Train the model
trainer.train()

model.save_pretrained("./prompt-tuning-fine-tuned-model")
tokenizer.save_pretrained("./prompt-tuning-fine-tuned-model")


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Trainable parameters: 15360
Total parameters: 1803482112
Percentage of trainable parameters: 0.00%


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-1d5f90ceabda>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: seyyed-msl82 (seyyed-msl8

Epoch,Training Loss,Validation Loss
1,38.816700,3.882910


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Fine-Tuned Model Testing:


NameError: name 'test_model' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Extract training and validation loss from the trainer state
train_loss = [log["loss"] for log in trainer.state.log_history if "loss" in log]
val_loss = [log["eval_loss"] for log in trainer.state.log_history if "eval_loss" in log]

# Plot the loss curves
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss (Prompt Tuning)")
plt.legend()
plt.show()

In [ ]:
import torch
from peft import PeftModel

# # Load the saved model, because the process is not integrated and encountered with several intruptions
# free_gpu_memory()

# # Load the LoRA fine-tuned model
# fine_tuned_model_path = "/kaggle/input/nndl-hwextra-q01/prompt-tuning-fine-tuned-model/"
# model = PeftModel.from_pretrained(model, fine_tuned_model_path)

# # Move model to GPU if available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# print("Model loaded successfully!")


In [21]:
val_inputs = [generate_prompt(example) for i, example in enumerate(val_dataset) if i < 100]
val_references = [example["response"] for i, example in enumerate(val_dataset) if i < 100]

# Generate predictions
val_predictions = generate_predictions(model, tokenizer, val_inputs, max_length=100)

# Compute ROUGE scores
rouge_scores = evaluate_rouge(val_predictions, val_references)
print("ROUGE Scores:", rouge_scores)

# Compute BERTScore
bert_scores = evaluate_bert_score(val_predictions, val_references)
print("BERTScore:", bert_scores)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

ROUGE Scores: {'rouge1': 0.12594712582426776, 'rouge2': 0.10505382891910336, 'rougeL': 0.1231549314786792, 'rougeLsum': 0.12323323186979929}
BERTScore: {'precision': 0.7780030965805054, 'recall': 0.8577991724014282, 'f1': 0.8131463527679443}


In [ ]:
free_gpu_memory()

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    device_map="auto", 
    use_auth_token=True
)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

lora_config = LoraConfig(
     r= 8, 
     lora_alpha=16,
     target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head'],
     lora_dropout=0.05,
     bias="none",
     task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./lora-results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=10,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./lora-logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    optim="paged_adamw_8bit"
)

free_gpu_memory()

lora_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

# Train the model
lora_trainer.train()

# Save the fine-tuned model
model.save_pretrained("./lora-fine-tuned-model")
tokenizer.save_pretrained("./lora-fine-tuned-model")

free_gpu_memory()

# Extract training and validation loss from the trainer state
train_loss = lora_trainer.state.log_history[-1]["loss"]
val_loss = lora_trainer.state.log_history[-1]["eval_loss"]

# Plot the loss curves
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()
free_gpu_memory()

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-4c20315f28e2>:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  lora_trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the 

Epoch,Training Loss,Validation Loss
1,17.416700,1.755975


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


KeyError: 'loss'

In [ ]:
train_loss = [log["loss"] for log in lora_trainer.state.log_history if "loss" in log]
val_loss = [log["eval_loss"] for log in lora_trainer.state.log_history if "eval_loss" in log]

# Plot the loss curves
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()


In [20]:
import torch
from peft import PeftModel

# # Load the saved model, because the process is not integrated and encountered with several intruptions
# free_gpu_memory()

# # Load the LoRA fine-tuned model
# fine_tuned_model_path = "/kaggle/input/nndl-hwextra-q01/lora-fine-tuned-model/"
# model = PeftModel.from_pretrained(model, fine_tuned_model_path)

# # Move model to GPU if available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# print("Model loaded successfully!")


Model loaded successfully!


/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0

In [ ]:
# Generate predictions
val_predictions = generate_predictions(model, tokenizer, val_inputs, max_length=100)

# Compute ROUGE scores
rouge_scores = evaluate_rouge(val_predictions, val_references)
print("ROUGE Scores:", rouge_scores)

# Compute BERTScore
bert_scores = evaluate_bert_score(val_predictions, val_references)
print("BERTScore:", bert_scores)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

ROUGE Scores: {'rouge1': 0.1286273279455361, 'rouge2': 0.10695067333690936, 'rougeL': 0.12615945496766048, 'rougeLsum': 0.1274506511474402}


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTScore: {'precision': 0.7793315649032593, 'recall': 0.8583532571792603, 'f1': 0.8138334155082703}


In [ ]:
free_gpu_memory()

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    use_auth_token=True
)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

# Freeze and Unfreeze Layers
# Extract the model's layers
layers = list(model.model.layers)

# Freeze all layers except the first and last layers
for i, layer in enumerate(layers):
    if i != 0 and i != len(layers) - 1:  # Freeze all layers except the first and last
        for param in layer.parameters():
            param.requires_grad = False


print("\nModel Structure Summary:")
print("=" * 60)
for i, layer in enumerate(layers):
    status = "Trainable" if any(param.requires_grad for param in layer.parameters()) else "Frozen"
    print(f"Layer {i}: {layer.__class__.__name__} - {status}")
print("=" * 60, "\n")

model.gradient_checkpointing_enable()

training_args = TrainingArguments(
    output_dir="./traditional-fine-tuning-results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./traditional-fine-tuning-logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    optim="adamw_hf"
)

free_gpu_memory()

traditional_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

# Train the model
traditional_trainer.train()

# Save the fine-tuned model
model.save_pretrained("./traditional-fine-tuned-model")
tokenizer.save_pretrained("./traditional-fine-tuned-model")


In [ ]:
train_loss = [log["loss"] for log in traditional_trainer.state.log_history if "loss" in log]
val_loss = [log["eval_loss"] for log in traditional_trainer.state.log_history if "eval_loss" in log]

# Plot the loss curves
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()


In [ ]:
# Generate predictions
val_predictions = generate_predictions(model, tokenizer, val_inputs, max_length=100)

# Compute ROUGE scores
rouge_scores = evaluate_rouge(val_predictions, val_references)
print("ROUGE Scores:", rouge_scores)

# Compute BERTScore
bert_scores = evaluate_bert_score(val_predictions, val_references)
print("BERTScore:", bert_scores)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for